# Uploading to Shared Google Drive

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Optiver_Challenge_Team_Base"
path_extra_input_data = f"{path}/Data"
# print(os.listdir(path))

# Install packages

In [2]:
!pip install fredapi

In [1]:
from fredapi import Fred
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import glob
import os

# # Connection to the API of the "https://fred.stlouisfed.org" website

In [2]:
fred = Fred(api_key = "XXX")

To find correspondances between categories names and their code, go on the website "https://fred.stlouisfed.org/categories" and when you select one category, you can find its code in the search bar.

=> 32455: Prices :
- 32217: Commodities
- 9: Consumer Price Indexes (CPI and PCE)
- 33913: Cryptocurrencies
- 4: Employment Cost Index
- 33717: Health Care Indexes
- 32261: House Price Indexes
- 31: Producer Price Indexes (PPI)
- 32220: Trade Indexes

=> 32991: Money, Banking, & Finance

=> 10: Population, Employment, & Labor Markets

=> 32992: National Accounts

=> 1: Production & Business Activity

=> 32263: International

=> 3008: U.S. Regional Data

=> 33060: Academic Data

# Extraction of specific series based on their code

- To see the different categories: "https://fred.stlouisfed.org/categories";
- To see Food series: "https://fred.stlouisfed.org/release/tables?rid=454&eid=816251"

In [3]:
series_dict = {
    # Food series
    # Alcoholic Beverages
    "Average Price: Malt Beverages, All Types, All Sizes, Any Origin (Cost per 16 Ounces/473.2 Milliliters) in U.S. City Average": "APU0000720111",
    "Average Price: Wine, Red and White Table, All Sizes, Any Origin (Cost per 1 Liter/33.8 Ounces) in U.S. City Average": "APU0000720311",
    # Beef and veal
    "Average Price: All Uncooked Beef Roasts (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000FC2101",
    "Average Price: All Uncooked Beef Steaks (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000FC3101",
    "Average Price: All Uncooked Other Beef (Excluding Veal) (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000FC4101", # missing data for April 2020
    "Average Price: All Uncooked Ground Beef (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000FC1101",
    # "Average Price: Beef for Stew, Boneless (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000703432", # missing data for January 2017 - October 2018 and May 2019
    "Average Price: Chuck Roast, USDA Choice, Boneless (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000703213",
    "Average Price: Ground Beef, 100% Beef (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000703112", # missing data for October 2012
    "Average Price: Ground Beef, Lean and Extra Lean (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000703113",
    "Average Price: Ground Chuck, 100% Beef (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000703111",
    "Average Price: Round Roast, USDA Choice, Boneless (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000703311",
    "Average Price: Steak, Round, USDA Choice, Boneless (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000703511",
    "Average Price: Steak, Sirloin, USDA Choice, Boneless (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000703613",
    # Cereals and bakery products
    "Average Price: Bread, White, Pan (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000702111",
    "Average Price: Bread, Whole Wheat, Pan (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000702212", # missing data for April 1988 - December 1990, June 2003, November 2003 - February 2004 and May 2004 - September 2004
    "Average Price: Cookies, Chocolate Chip (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000702421",
    "Average Price: Flour, White, All Purpose (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000701111", # missing data for April 2020
    "Average Price: Rice, White, Long Grain, Uncooked (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000701312", # missing data for May 2000 - April 2002 and April 2020
    "Average Price: Spaghetti and Macaroni (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000701322",
    # Dairy products
    "Average Price: American Processed Cheese (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000710211", # missing data for February 1987 and November 2018
    "Average Price: Cheddar Cheese, Natural (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000710212", # missing data for Mars 1989 - April 1989, December 1989 - December 1990
    "Average Price: Ice Cream, Prepackaged, Bulk, Regular, per One-Half Gallon (1.9 Liters) in U.S. City Average": "APU0000710411", # missing data for January 1987
    # "Average Price: Milk, Fresh, Low-Fat, Reduced Fat, Skim (Cost per Gallon/3.8 Liters) in U.S. City Average": "APU0000FJ1101", # No data before April 2018
    "Average Price: Milk, Fresh, Whole, Fortified (Cost per Gallon/3.8 Liters) in U.S. City Average": "APU0000709112",
    # "Average Price: Yogurt (Cost per 8 Ounces/226.8 Grams) in U.S. City Average": "APU0000FJ4101", # No data before April 2018
    # Fresh fruits and vegetables
    "Average Price: Bananas (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000711211", # missing data for January 1987 - February 1987
    # "Average Price: Grapefruit (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000711411", # missing data for October 2005, September 2016 - October 2016, March 2020 - May 2023 and October 2024
    # "Average Price: Lemons (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000711412", # many missing data between March 2020 and February 2022
    # "Average Price: Lettuce, Iceberg (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000712211", # missing data for March 2020 - April 2023
    # "Average Price: Oranges, Navel (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000711311", # many missing data before August 2022
    "Average Price: Potatoes, White (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000712112", # missing data for March 2020 - April 2020
    "Average Price: Strawberries, Dry Pint (Cost per 12 Ounces/340.2 Grams) in U.S. City Average": "APU0000711415", # missing data for October 2013, November 2015, April 2020, December 2021 and many missing data before January 2004
    "Average Price: Tomatoes, Field Grown (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000712311", # missing data for January 1987 - Fabruary 1987, October 2013 and March 2020
    # Other food items
    # "Average Price: All Soft Drinks, 12 Pk, 12 Ounce, Cans (Cost per 12 Ounces/354.9 Milliliters) in U.S. City Average": "APU0000FN1102", # No data before April 2018
    # "Average Price: All Soft Drinks (Cost per 2 Liters/67.6 Ounces) in U.S. City Average": "APU0000FN1101", # No data before April 2018
    # "Average Price: Butter, Stick (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000FS1101", # No data before April 2018
    # "Average Price: Coffee, 100%, Ground Roast, All Sizes (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000717311", # missing data for October 2007, January 2008 - November 2009, June 2018 - August 2018 and October 2018 - September 2019
    "Average Price: Potato Chips (Cost per 16 Ounces) in U.S. City Average": "APU0000718311",
    "Average Price: Sugar, White, All Sizes (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000715211",
    # Pork and other meats
    "Average Price: All Ham (Excluding Canned Ham and Luncheon Slices) (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000FD2101", # missing data for April 2020 - September 2020
    "Average Price: All Other Pork (Excluding Canned Ham and Luncheon Slices) (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000FD4101", # missing data for April 2020
    "Average Price: All Pork Chops (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000FD3101",
    "Average Price: Bacon, Sliced (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000704111",
    "Average Price: Chops, Boneless (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000704212",
    "Average Price: Chops, Center Cut, Bone-In (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000704211",
    "Average Price: Ham, Boneless, Excluding Canned (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000704312", # missing data for April 2020
    # Poultry, fish, and eggs
    "Average Price: Chicken Breast, Boneless (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000FF1101",
    "Average Price: Chicken Legs, Bone-In (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000706212", # missing data for January 1987
    "Average Price: Chicken, Fresh, Whole (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000706111", # missing data for May 2020
    "Average Price: Eggs, Grade A, Large (Cost per Dozen) in U.S. City Average": "APU0000708111", # Our target variable (also called dependent variable)
    # Processed fruits and vegetables
    "Average Price: Beans, Dried, Any Type, All Sizes (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000714233", # missing data for April 2020 - May 2020
    # "Average Price: Corn, Canned, Any Style, All Sizes (Cost per Pound/453.6 Grams) in U.S. City Average": "APU0000714221", # Almost no data before October 2023
    "Average Price: Orange Juice, Frozen Concentrate, 12 Ounce Can (Cost per 16 Ounces/473.2 Milliliters) in U.S. City Average": "APU0000713111",

    # Categories => Prices => Commodities
    "Global price of Soybean Meal": "PSMEAUSDM",
    "Global price of Corn": "PMAIZMTUSDM",
    "Global price of Wheat": "PWHEAMTUSDM",
    "Global price of Barley": "PBARLUSDM",
    "Global price of Soybean Meal": "PSMEAUSDM",
    "Global price of Rapeseed Oil": "PROILUSDM",
    "Global price of Poultry": "PPOULTUSDM",

    # Categories => Prices => Producer Price Indexes (PPI) => Commodity Based => Farm Products
    "Producer Price Index by Commodity: Farm Products: Oats": "WPU012203",

    # Categories => National Income & Product Accounts => Personal Income & Outlays => Personal Consumption Expenditures
    # Maybe use log() to scale the variable, because it's in Billions of Dollars
    "Personal consumption expenditures: Food": "DFXARC1M027SBEA",
    "Personal Consumption Expenditures: Durable Goods": "PCEDG",
    "Personal Consumption Expenditures: Services": "PCES",
    "Personal Consumption Expenditures: Nondurable Goods": "PCEND"
}

This is the loop wich extract each series by iterating in each element of the "series_dict" dictionnaire.

In [4]:
df = pd.DataFrame()

for name, series_id in tqdm(series_dict.items(), total=len(series_dict), desc="Fetching FRED Series Data"):
    series_data = fred.get_series(series_id)
    series_data = series_data.rename(name)
    df = pd.concat([df, series_data], axis=1, join="outer")

df.index = pd.to_datetime(df.index)
df = df.sort_index()

Fetching FRED Series Data: 100%|██████████| 53/53 [00:22<00:00,  2.31it/s]


In [5]:
print(f"The exctracted dataframe contains {df.shape[0]} rows and {df.shape[1]} columns.")
df.head()

The exctracted dataframe contains 793 rows and 53 columns.


,"Average Price: Malt Beverages, All Types, All Sizes, Any Origin (Cost per 16 Ounces/473.2 Milliliters) in U.S. City Average","Average Price: Wine, Red and White Table, All Sizes, Any Origin (Cost per 1 Liter/33.8 Ounces) in U.S. City Average",Average Price: All Uncooked Beef Roasts (Cost per Pound/453.6 Grams) in U.S. City Average,Average Price: All Uncooked Beef Steaks (Cost per Pound/453.6 Grams) in U.S. City Average,Average Price: All Uncooked Other Beef (Excluding Veal) (Cost per Pound/453.6 Grams) in U.S. City Average,Average Price: All Uncooked Ground Beef (Cost per Pound/453.6 Grams) in U.S. City Average,"Average Price: Chuck Roast, USDA Choice, Boneless (Cost per Pound/453.6 Grams) in U.S. City Average","Average Price: Ground Beef, 100% Beef (Cost per Pound/453.6 Grams) in U.S. City Average","Average Price: Ground Beef, Lean and Extra Lean (Cost per Pound/453.6 Grams) in U.S. City Average","Average Price: Ground Chuck, 100% Beef (Cost per Pound/453.6 Grams) in U.S. City Average",...,Global price of Corn,Global price of Wheat,Global price of Barley,Global price of Rapeseed Oil,Global price of Poultry,Producer Price Index by Commodity: Farm Products: Oats,Personal consumption expenditures: Food,Personal Consumption Expenditures: Durable Goods,Personal Consumption Expenditures: Services,Personal Consumption Expenditures: Nondurable Goods
1959-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,61.9,42.3,138.7,125.1
1959-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,61.4,44.2,140.0,125.4
1959-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,60.5,44.4,140.5,127.7
1959-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,60.3,45.1,141.5,125.6
1959-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,61.6,45.4,142.8,128.0


Function to check that our series have monthly frequency.

In [6]:
def verif_freq_time_series(dtf):
  """Verify frequency consistency in merged time series data exctracted.

  The exctracted DataFrames are created by concatenating individual time series
  using `pd.concat(...)`, aligning them based on dates.  If the source series have different
  frequencies (e.g., one daily, another monthly), the resulting DataFrame will reflect this.
  Monthly series will only have data points for the first of each month where overlapping
  data exists. Other days in the month will show NaN values for these monthly series. This
  code block checks for this behaviour."""

  dtf['Year'] = dtf.index.year
  yearly_counts = dtf.groupby('Year').size()
  print(f"The years for which the number of datapoints is different than 12 are: {yearly_counts[yearly_counts != 12]}")

  unexpected_days = dtf.index[dtf.index.day != 1]
  print(f"The dates for which the month day is not the first: {dtf.loc[unexpected_days]}")

  print(f"The frequency is: {pd.infer_freq(dtf.index)}")

In [7]:
verif_freq_time_series(df)

The years for which the number of datapoints is different than 12 are: Year
2025    1
dtype: int64
The dates for which the month day is not the first: Empty DataFrame
Columns: [Average Price: Malt Beverages, All Types, All Sizes, Any Origin (Cost per 16 Ounces/473.2 Milliliters) in U.S. City Average, Average Price: Wine, Red and White Table, All Sizes, Any Origin (Cost per 1 Liter/33.8 Ounces) in U.S. City Average, Average Price: All Uncooked Beef Roasts (Cost per Pound/453.6 Grams) in U.S. City Average, Average Price: All Uncooked Beef Steaks (Cost per Pound/453.6 Grams) in U.S. City Average, Average Price: All Uncooked Other Beef (Excluding Veal) (Cost per Pound/453.6 Grams) in U.S. City Average, Average Price: All Uncooked Ground Beef (Cost per Pound/453.6 Grams) in U.S. City Average, Average Price: Chuck Roast, USDA Choice, Boneless (Cost per Pound/453.6 Grams) in U.S. City Average, Average Price: Ground Beef, 100% Beef (Cost per Pound/453.6 Grams) in U.S. City Average, Average Pri

# Missing Values

In [8]:
filtered_df = df[df.index >= pd.Timestamp('2006-01-01')]

In [7]:
def check_missing_value(df):
    # Number of missing values for each column
    missing_values_count = df.isnull().sum()
    print(missing_values_count)

    # Indices of missing values for each column
    missing_values_indices = {col: df[col].isnull().where(df[col].isnull() == True).dropna().index.tolist() for col in df.columns}
    for col, indices in missing_values_indices.items():
        print(f"Column: {col}, Missing Indices: {indices}")

In [10]:
check_missing_value(filtered_df)

Average Price: Malt Beverages, All Types, All Sizes, Any Origin (Cost per 16 Ounces/473.2 Milliliters) in U.S. City Average    0
Average Price: Wine, Red and White Table, All Sizes, Any Origin (Cost per 1 Liter/33.8 Ounces) in U.S. City Average            0
Average Price: All Uncooked Beef Roasts (Cost per Pound/453.6 Grams) in U.S. City Average                                      0
Average Price: All Uncooked Beef Steaks (Cost per Pound/453.6 Grams) in U.S. City Average                                      0
Average Price: All Uncooked Other Beef (Excluding Veal) (Cost per Pound/453.6 Grams) in U.S. City Average                      1
Average Price: All Uncooked Ground Beef (Cost per Pound/453.6 Grams) in U.S. City Average                                      0
Average Price: Chuck Roast, USDA Choice, Boneless (Cost per Pound/453.6 Grams) in U.S. City Average                            0
Average Price: Ground Beef, 100% Beef (Cost per Pound/453.6 Grams) in U.S. City Average          

In [11]:
filtered_df_filled = filtered_df.copy()

last_indice = filtered_df.index[-1:]

filtered_df_filled.loc[~filtered_df_filled.index.isin(last_indice)] = (
    filtered_df_filled.loc[~filtered_df_filled.index.isin(last_indice)].interpolate(method='linear')
)

In [12]:
check_missing_value(filtered_df_filled)

Average Price: Malt Beverages, All Types, All Sizes, Any Origin (Cost per 16 Ounces/473.2 Milliliters) in U.S. City Average    0
Average Price: Wine, Red and White Table, All Sizes, Any Origin (Cost per 1 Liter/33.8 Ounces) in U.S. City Average            0
Average Price: All Uncooked Beef Roasts (Cost per Pound/453.6 Grams) in U.S. City Average                                      0
Average Price: All Uncooked Beef Steaks (Cost per Pound/453.6 Grams) in U.S. City Average                                      0
Average Price: All Uncooked Other Beef (Excluding Veal) (Cost per Pound/453.6 Grams) in U.S. City Average                      0
Average Price: All Uncooked Ground Beef (Cost per Pound/453.6 Grams) in U.S. City Average                                      0
Average Price: Chuck Roast, USDA Choice, Boneless (Cost per Pound/453.6 Grams) in U.S. City Average                            0
Average Price: Ground Beef, 100% Beef (Cost per Pound/453.6 Grams) in U.S. City Average          

Extract the DataSet

In [37]:
# Extraction of the dataframe containing the time series into a .csv file, and upload it on the Google Drive
csv_filename = "FredStlouis_Selected_Series.csv"
csv_path = os.path.join(f"{path_extra_input_data}/FRED", csv_filename)
df.to_csv(csv_path, index=True)

In [13]:
filtered_df_filled.to_csv("FRED/FredStlouis_Selected_Series.csv", index=True)

# Extraction of the entire category

In [ ]:
desired_category = 32217
# Here we create a dataframe named "df_category" containing all the series in the category, along with their respective informations
df_category = fred.search_by_category(desired_category)
# We filter to have only the series having data up to the first of january 2025 and a minimum of history of 2 years (i.e. data before the first january 2023)
df_category = df_category[(df_category["observation_end"] == pd.Timestamp("2025-01-01")) &
                 (df_category["observation_start"] <= pd.Timestamp("2023-01-01"))] # (df["frequency"] == "Monthly")
df_category.head()

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
series id,,,,,,,,,,,,,,,
APU0000701111,APU0000701111,2025-02-17,2025-02-17,"Average Price: Flour, White, All Purpose (Cost...",1980-01-01,2025-01-01,Monthly,M,U.S. Dollars,U.S. $,Not Seasonally Adjusted,NSA,2025-02-12 07:34:01-06:00,56,"All white, all-purpose flour regardless of pac..."
APU0000701312,APU0000701312,2025-02-17,2025-02-17,"Average Price: Rice, White, Long Grain, Uncook...",1980-01-01,2025-01-01,Monthly,M,U.S. Dollars,U.S. $,Not Seasonally Adjusted,NSA,2025-02-12 07:34:02-06:00,45,"All white, long-grain, uncooked rice regardles..."
APU0000701322,APU0000701322,2025-02-17,2025-02-17,Average Price: Spaghetti and Macaroni (Cost pe...,1984-01-01,2025-01-01,Monthly,M,U.S. Dollars,U.S. $,Not Seasonally Adjusted,NSA,2025-02-12 07:34:02-06:00,29,Include all long thin pasta except fettuccine ...
APU0000702111,APU0000702111,2025-02-17,2025-02-17,"Average Price: Bread, White, Pan (Cost per Pou...",1980-01-01,2025-01-01,Monthly,M,U.S. Dollars,U.S. $,Not Seasonally Adjusted,NSA,2025-02-12 07:34:02-06:00,68,All white pre-packaged regular bread regardles...
APU0000702212,APU0000702212,2025-02-17,2025-02-17,"Average Price: Bread, Whole Wheat, Pan (Cost p...",1980-01-01,2025-01-01,Monthly,M,U.S. Dollars,U.S. $,Not Seasonally Adjusted,NSA,2025-02-12 07:34:03-06:00,32,"100% whole wheat pan-baked bread, regardless o..."


In [ ]:
# We use the previous "df_category" dataframe to iterate over each of its rows, select the row's "series id" and finally extract the associated time series data.
merged_df_category = pd.DataFrame()
index_line = 0


for fred_id, row in tqdm(df_category.iterrows(), total=len(df_category), desc="Fetching FRED Data"):
  series_title = row["title"]

  try:
    series_data = fred.get_series(fred_id)

    series_data = series_data.to_frame(name=series_title)

    merged_df_category = pd.concat([merged_df_category, series_data], axis=1)

    if index_line % 50 == 0 and index_line != 0:
        time.sleep(20) # this is to make a "pause" at each 50 rows because if not the API bloc the extraction with a "To many requests" error

    index_line += 1

  except Exception as e:
    print(f"Error fetching {series_title} (ID: {fred_id}): {e}")


print(merged_df_category.shape)
merged_df_category

Fetching FRED Data: 100%|██████████| 533/533 [06:04<00:00,  1.46it/s]


In [ ]:
# Verify frequency consistency in merged time series data.
verif_freq_time_series(merged_df_category)

In [ ]:
# Extraction of the dataframe coontaining the time series into a .csv file, and upload it on the Google Drive
csv_filename = "commodities_32217_before2023_until2025.csv"
csv_path = os.path.join(f"{path_extra_input_data}/FRED", csv_filename)
merged_df_category.to_csv(csv_path, index=True)

# Non FRED data

Here I just took the other data we choosed, keept only the columns of interest (dates/values), checked for missing data and created new .csv files of these clean datas.

In [135]:
# df = pd.read_csv(f"{path}/Data/Google_Trends_Wikipedia_GDelt/gdelt.2016-25.FOOD.AGRIC.INF.LIVESTO.COMDTIES.SUPCHAIN_articlecount-avgtone.csv")
df = pd.read_csv("old_Google_Trends_Wikipedia_GDelt/wikipageviews_PRICEOFEGGS_2015_2025.csv")

print(f'''Shape of the dataframe: {df.shape[0]} lines and {df.shape[1]} columns.
Column's names of the dataframe: {df.columns}.
Type of the values in the columns: {df.dtypes}.
Five first rows of the dataframe:''')
df.head()

Shape of the dataframe: 115 lines and 2 columns.
Column's names of the dataframe: Index(['Date', 'Price of eggs'], dtype='object').
Type of the values in the columns: Date             object
Price of eggs     int64
dtype: object.
Five first rows of the dataframe:


,Date,Price of eggs
0,2015-07,304
1,2015-08,274
2,2015-09,294
3,2015-10,267
4,2015-11,252


In [138]:
type(df.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [137]:
# df['date'] = pd.to_datetime(df['Year'].astype(str) + df['Period'], format='%Y%b')
# df['date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
# df = df.iloc[1:, :]
df.index = pd.to_datetime(df.index, format='%Y-%m')
df.index.name = None
df.sort_index(inplace=True)
df.head()

,Price of eggs
2015-07-01,304
2015-08-01,274
2015-09-01,294
2015-10-01,267
2015-11-01,252


In [139]:
# columns_to_keep = ["article_count", "avg_tone"]
# df = df[columns_to_keep]
# df["Value"] = df['Value'].str.replace(',', '').astype(int)
# df["Value"] = df['Value'].astype(float)
# df["Value"] = df['Value'].str.replace(',', '.').astype(float)
# df["Category: All categories"] = df['Category: All categories'].str.replace('<', '').astype(int)
df.rename(columns={"Price of eggs": "Wiki_Page_View_Price_of_Eggs",
                   }, inplace=True)
df.head()

,Wiki_Page_View_Price_of_Eggs
2015-07-01,304
2015-08-01,274
2015-09-01,294
2015-10-01,267
2015-11-01,252


In [ ]:
# Extraction of the dataframe coontaining the time series into a .csv file, and upload it on the Google Drive
csv_filename = "Gle_Trend_Brid_Flu_2004_2025.csv"
csv_path = os.path.join(f"{path_extra_input_data}/FRED", csv_filename)
df.to_csv(csv_path, index=True)

In [140]:
df.to_csv("Google_Trends_Wikipedia_GDelt/Wiki_Page_View_Price_of_Eggs_2015_2025.csv", index=True)

# Merging all datasets into one

In [4]:
csv_files = glob.glob("**/*.csv", recursive=True)

# Initialize an empty list to store DataFrames
dfs = []

# Read each CSV file and append to the list
for file in csv_files:
    df = pd.read_csv(file, index_col=0, parse_dates=True)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, axis=1, join='outer')

combined_df.head()

,Gle_Trend_Brid_Flu,Wiki_Page_View_Eggs_Substitutes,Gle_Trend_Chicken_Feed_Price,Gle_Trend_Eggs_Prices,Gle_Trend_Eggs_Shortage,Wiki_Page_View_Bird_Flu,Wiki_Page_View_Price_of_Eggs,GDelt_Article_Count_seach_term_food_agri_livestock_infla_commo_supplychain,GDelt_Sentiment_seach_term_food_agri_livestock_infla_commo_supplychain,Rate_of_Lay_Eggs_100_Layer,...,Global price of Wheat,Global price of Barley,Global price of Rapeseed Oil,Global price of Poultry,Producer Price Index by Commodity: Farm Products: Oats,Personal consumption expenditures: Food,Personal Consumption Expenditures: Durable Goods,Personal Consumption Expenditures: Services,Personal Consumption Expenditures: Nondurable Goods,Year
1948-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1948-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1948-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1948-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1948-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
combined_df_filtered = combined_df[combined_df.index >= pd.Timestamp('2012-01-01')]

In [11]:
check_missing_value(combined_df_filtered)

Gle_Trend_Brid_Flu                                      0
Wiki_Page_View_Eggs_Substitutes                        43
Gle_Trend_Chicken_Feed_Price                            0
Gle_Trend_Eggs_Prices                                   0
Gle_Trend_Eggs_Shortage                                 0
                                                       ..
Personal consumption expenditures: Food                 2
Personal Consumption Expenditures: Durable Goods        2
Personal Consumption Expenditures: Services             2
Personal Consumption Expenditures: Nondurable Goods     2
Year                                                    1
Length: 66, dtype: int64
Column: Gle_Trend_Brid_Flu, Missing Indices: []
Column: Wiki_Page_View_Eggs_Substitutes, Missing Indices: [Timestamp('2012-01-01 00:00:00'), Timestamp('2012-02-01 00:00:00'), Timestamp('2012-03-01 00:00:00'), Timestamp('2012-04-01 00:00:00'), Timestamp('2012-05-01 00:00:00'), Timestamp('2012-06-01 00:00:00'), Timestamp('2012-07-01 00:0

In [12]:
combined_df_filtered.to_csv("Final_Dataset/Predict_Avg_Eggs_Price_Dataset.csv", index=True)